In [1]:
import uxarray as ux
import xarray as xr
import numpy as np

In [2]:
from geocat.comp.meteorology import delta_pressure


In [3]:
ux.__version__


'2025.10.0'

In [ ]:
uxds = ux.open_dataset("../test/grid.nc", "../test/data.nc")

uxds = uxds.rename({'t2m': 'V'})
plevs = [10000, 5000, 1000]
uxds['V'] = uxds['V'].expand_dims({'plev': plevs})
uxds = uxds.assign_coords(plev=('plev', plevs))
uxds['PS'] = uxds['V'] * 0.001 + 10000
uxds

<xarray.UxDataset> Size: 120B
Dimensions:  (plev: 3, n_face: 4)
Coordinates:
  * plev     (plev) int64 24B 100000 50000 10000
Dimensions without coordinates: n_face
Data variables:
    V        (plev, n_face) float32 48B 297.6 297.6 297.7 ... 297.6 297.7 297.3
    PS       (plev, n_face) float32 48B 1e+04 1e+04 1e+04 ... 1e+04 1e+04 1e+04

In [21]:
def zonal_mpsi(uxds):
    """
    Calculate the zonally averaged meridional mass streamfunction (mpsi) from a UXarray Dataset.

    Parameters
    ----------
    uxds : uxarray.UxDataset
        Input dataset containing the following required fields:
            - V : Meridional wind component (on pressure or hybrid sigma-pressure levels)
            - PS : Surface pressure
            - plev : Pressure levels (if V is on pressure levels)
            - hyam : Hybrid A coefficients (if V is on hybrid sigma-pressure levels)
            - hybm : Hybrid B coefficients (if V is on hybrid sigma-pressure levels)
            - uxgrid : Grid information for uxarray

    Returns
    -------
    ux_mpsi : uxarray.UxDataArray
        Zonal mean meridional mass streamfunction, scaled by Earth's geometry and gravity.
        Dimensions: ["time", "latitudes", "plev"]

    Notes
    -----
    - Optionally converts wind data from hybrid sigma-pressure levels to pressure levels.
    - Computes zonal means and pressure integration deltas.
    - Integrates over pressure levels, handling orientation.
    - Applies scaling factor based on Earth's radius and gravity.
    - Currently only supports UXarray Datasets with specified structure.
    """

    # constants
    a = 6.371e6  # Earth radius (m)
    g = 9.80665  # gravity (m/s^2)

    # Basic input validation
    if not hasattr(uxds, "V") or not hasattr(uxds, "PS"):
        raise AttributeError(
            "zonal_mpsi: input uxds must contain 'V' and 'PS' variables"
        )

    if not hasattr(uxds, "uxgrid"):
        raise AttributeError(
            "zonal_mpsi: input uxds missing required 'uxgrid' metadata (uxarray)"
        )

    # Auto-detect whether V is already on pressure levels (prefer explicit 'plev').
    is_on_plev = False
    if "plev" in getattr(uxds.V, "coords", {}) or "plev" in getattr(uxds.V, "dims", ()):
        is_on_plev = True

    if is_on_plev:
        # accept UxDataArray or DataArray inputs
        if isinstance(uxds.V, ux.UxDataArray):
            da_ipress = uxds.V
        else:
            da_ipress = xr.DataArray(uxds.V)
        # Only pass a ux.Grid instance to UxDataArray; otherwise pass None
        _uxgrid = getattr(uxds, "uxgrid", None)
        grid_param = (
            _uxgrid if (hasattr(ux, "Grid") and isinstance(_uxgrid, ux.Grid)) else None
        )
        
        ux_ipress = ux.UxDataArray(da_ipress, uxgrid=grid_param)
    else:
        # require hybrid coefficients
        if not hasattr(uxds, "hyam") or not hasattr(uxds, "hybm"):
            raise AttributeError(
                "zonal_mpsi: input uxds must have either a 'plev' coordinate on V or hybrid coefficients 'hyam' and 'hybm'"
            )

        # determine a reasonable lev_dim to pass to the interpolator (use the last dim of V)
        lev_dim = (
            uxds.V.dims[-1]
            if hasattr(uxds.V, "dims") and len(uxds.V.dims) > 0
            else None
        )

        try:
            da_ipress = interp_hybrid_to_pressure(
                uxds.V, uxds.PS, uxds.hyam, uxds.hybm, lev_dim=lev_dim
            )
        except Exception as e:
            raise RuntimeError(
                f"zonal_mpsi: failed to convert hybrid levels to pressure levels: {e}"
            )

        # Ensure we have an xarray DataArray and convert to uxarray UxDataArray
        if not isinstance(da_ipress, xr.DataArray):
            da_ipress = xr.DataArray(da_ipress)

    _uxgrid = getattr(uxds, "uxgrid", None)
    grid_param = (
        _uxgrid if (hasattr(ux, "Grid") and isinstance(_uxgrid, ux.Grid)) else None
    )
    ux_ipress = ux.UxDataArray(da_ipress, uxgrid=grid_param)

    # Require a canonical 'plev' coordinate on the pressure-level data.
    if "plev" not in ux_ipress.coords and "plev" not in ux_ipress.dims:
        raise ValueError(
            "zonal_mpsi: could not find pressure-level coordinate 'plev' in converted data; provide 'plev' or hybrid coefficients"
        )

    # Keep UX types up through the zonal_mean step, Xarray after
    da_v_zonal = ux_ipress.zonal_mean()

    # Zonal mean of surface pressure
    if hasattr(uxds.PS, "zonal_mean"):
        ux_PS_zonal = uxds.PS.zonal_mean()
    else:
        # infer a longitude dim name
        lon_dim = None
        for d in uxds.PS.dims:
            if "lon" in d or "long" in d:
                lon_dim = d
                break
        if lon_dim is None:
            raise ValueError(
                "zonal_mpsi: cannot compute zonal mean of PS; no longitude dimension found"
            )
        ux_PS_zonal = uxds.PS.mean(dim=lon_dim)
    # integration deltas
    plev_coord = da_v_zonal.coords['plev']
    dp = delta_pressure(plev_coord, ux_PS_zonal)

    # Convert delta_pressure result to an xarray DataArray
    if isinstance(dp, xr.DataArray):
        da_dp_zonal = dp
    else:
        # infer dims and coords from da_v_zonal
        dims = da_v_zonal.dims
        coords = {d: da_v_zonal.coords[d] for d in dims if d in da_v_zonal.coords}
        coords['plev'] = da_v_zonal.coords['plev']
        da_dp_zonal = xr.DataArray(dp, dims=dims, coords=coords, name="delta_pressure")

    # scaling factor
    if "latitudes" not in da_v_zonal.coords:
        lat_candidates = ("lat", "latitude", "latitudes")
        lat_name = next((n for n in lat_candidates if n in da_v_zonal.coords), None)
        if lat_name is None:
            raise ValueError(
                "zonal_mpsi: could not find latitude coordinate on zonal data"
            )
    else:
        lat_name = "latitudes"

    da_scaling_factor = xr.DataArray(
        2 * np.pi * a * np.cos(da_v_zonal.coords[lat_name]) / g,
        coords={lat_name: da_v_zonal.coords[lat_name]},
        dims=(lat_name,),
    )

    # check orientation and integrate along pressure dim
    plev_values = da_v_zonal.coords["plev"].values
    increasing = plev_values[0] < plev_values[-1]

    if increasing:
        integrand = da_v_zonal * da_dp_zonal
        da_mpsi = integrand.cumsum(dim="plev")
    else:
        integrand = (da_v_zonal * da_dp_zonal).isel({"plev": slice(None, None, -1)})
        da_mpsi = integrand.cumsum(dim="plev").isel({"plev": slice(None, None, -1)})

    # apply scaling factor
    da_mpsi = da_mpsi * da_scaling_factor

    # metadata
    try:
        da_mpsi.attrs.update(
            {
                "long_name": "zonal mean meridional mass streamfunction",
                "units": "kg/s",
                "info": "zonal_mpsi: integrated meridional mass streamfunction",
            }
        )
    except Exception:
        pass

    return da_mpsi



In [22]:
zonal_mpsi(uxds)

/Users/jkent/miniconda3/envs/geocat_comp_build/lib/python3.13/site-packages/uxarray/core/zonal.py:66: RuntimeWarning: invalid value encountered in divide
  weighted = (data_slice * w_reshaped).sum(axis=face_axis) / total


/Users/jkent/miniconda3/envs/geocat_comp_build/lib/python3.13/site-packages/uxarray/core/zonal.py:66: RuntimeWarning: invalid value encountered in divide
  weighted = (data_slice * w_reshaped).sum(axis=face_axis) / total


ValueError: Zonal mean computations are currently only supported for face-centered data variables.

In [30]:
a = 6.371e6  # Earth radius (m)
g = 9.80665  # gravity (m/s^2)

# Basic input validation
if not hasattr(uxds, "V") or not hasattr(uxds, "PS"):
    raise AttributeError(
        "zonal_mpsi: input uxds must contain 'V' and 'PS' variables"
    )

if not hasattr(uxds, "uxgrid"):
    raise AttributeError(
        "zonal_mpsi: input uxds missing required 'uxgrid' metadata (uxarray)"
    )

# Auto-detect whether V is already on pressure levels (prefer explicit 'plev').
is_on_plev = False
if "plev" in getattr(uxds.V, "coords", {}) or "plev" in getattr(uxds.V, "dims", ()):
    is_on_plev = True

if is_on_plev:
    # accept UxDataArray or DataArray inputs
    if isinstance(uxds.V, ux.UxDataArray):
        da_ipress = uxds.V
    else:
        da_ipress = xr.DataArray(uxds.V)
    # Only pass a ux.Grid instance to UxDataArray; otherwise pass None
    _uxgrid = getattr(uxds, "uxgrid", None)
    grid_param = (
        _uxgrid if (hasattr(ux, "Grid") and isinstance(_uxgrid, ux.Grid)) else None
    )
        
    ux_ipress = ux.UxDataArray(da_ipress, uxgrid=grid_param)
else:
    # require hybrid coefficients
    if not hasattr(uxds, "hyam") or not hasattr(uxds, "hybm"):
        raise AttributeError(
            "zonal_mpsi: input uxds must have either a 'plev' coordinate on V or hybrid coefficients 'hyam' and 'hybm'"
        )

    # determine a reasonable lev_dim to pass to the interpolator (use the last dim of V)
    lev_dim = (
        uxds.V.dims[-1]
        if hasattr(uxds.V, "dims") and len(uxds.V.dims) > 0
        else None
    )

    try:
        da_ipress = interp_hybrid_to_pressure(
            uxds.V, uxds.PS, uxds.hyam, uxds.hybm, lev_dim=lev_dim
        )
    except Exception as e:
        raise RuntimeError(
            f"zonal_mpsi: failed to convert hybrid levels to pressure levels: {e}"
        )

    # Ensure we have an xarray DataArray and convert to uxarray UxDataArray
    if not isinstance(da_ipress, xr.DataArray):
        da_ipress = xr.DataArray(da_ipress)

_uxgrid = getattr(uxds, "uxgrid", None)
grid_param = (
    _uxgrid if (hasattr(ux, "Grid") and isinstance(_uxgrid, ux.Grid)) else None
)
ux_ipress = ux.UxDataArray(da_ipress, uxgrid=grid_param)

# Require a canonical 'plev' coordinate on the pressure-level data.
if "plev" not in ux_ipress.coords and "plev" not in ux_ipress.dims:
    raise ValueError(
        "zonal_mpsi: could not find pressure-level coordinate 'plev' in converted data; provide 'plev' or hybrid coefficients"
    )

# Keep UX types up through the zonal_mean step, Xarray after
da_v_zonal = ux_ipress.zonal_mean()

# Zonal mean of surface pressure
if hasattr(uxds.PS, "zonal_mean"):
    ux_PS_zonal = uxds.PS.zonal_mean()
else:
    # infer a longitude dim name
    lon_dim = None
    for d in uxds.PS.dims:
        if "lon" in d or "long" in d:
            lon_dim = d
            break
    if lon_dim is None:
        raise ValueError(
            "zonal_mpsi: cannot compute zonal mean of PS; no longitude dimension found"
        )
    ux_PS_zonal = uxds.PS.mean(dim=lon_dim)
# integration deltas
plev_coord = da_v_zonal.coords['plev']
dp = delta_pressure(plev_coord, ux_PS_zonal)

/Users/jkent/miniconda3/envs/geocat_comp_build/lib/python3.13/site-packages/uxarray/core/zonal.py:66: RuntimeWarning: invalid value encountered in divide
  weighted = (data_slice * w_reshaped).sum(axis=face_axis) / total


ValueError: min() iterable argument is empty